In [26]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.signal as signal
from synchronisation import *
import pyaudio
import time



In [22]:
def generate_sound(samples, volume, fs): # volume range [0.0, 1.0]

    p = pyaudio.PyAudio()
    time.sleep(1)

    output_bytes = (volume * samples).tobytes()

    # for paFloat32 sample values must be in range [-1.0, 1.0]
    stream = p.open(format=pyaudio.paFloat32,
                    channels=1,
                    rate=fs,
                    output=True)

    # play. May repeat with different volume values (if done interactively)
    start_time = time.time()
    stream.write(output_bytes)
    print("Played sound for {:.2f} seconds".format(time.time() - start_time))

    stream.stop_stream()
    stream.close()

    p.terminate()

In [23]:
fs = 44100
chirp_time = 1
t = np.linspace(0, chirp_time, fs*chirp_time)
f_0 = 1000
f_1 = 16000
chirp = signal.chirp(t, f0=f_0, f1=f_1, t1=chirp_time, method='linear')
chirp = chirp / np.max(np.abs(chirp))

symbol = []
for i in range(511):
    if i % 4 == 0:
        symbol.append(1+1j)
    elif i % 4 == 1:
        symbol.append(-1+1j)
    elif i % 4 == 2:
        symbol.append(1-1j)
    elif i % 4 == 3:
        symbol.append(-1-1j)


symbol = np.array(symbol)
conj_symbol = np.conj(symbol)[::-1]
symbol = np.concatenate((symbol, conj_symbol))
symbol = np.insert(symbol, 0, 0)
symbol = np.insert(symbol, 512, 0)
print(symbol.shape)
time_symbol = np.fft.ifft(symbol)
print(time_symbol.shape)
time_symbol = np.real(time_symbol)
time_symbol = time_symbol / np.max(np.abs(time_symbol))
cyclic = np.concatenate((time_symbol[-32:], time_symbol))

symbol_data = []
for i in range(511):
    if i % 4 == 3:
        symbol_data.append(1+1j)
    elif i % 4 == 2:
        symbol_data.append(-1+1j)
    elif i % 4 == 1:
        symbol_data.append(1-1j)
    elif i % 4 == 0:
        symbol_data.append(-1-1j)

symbol_data = np.array(symbol_data)
conj_symbol_data = np.conj(symbol_data)[::-1]
symbol_data = np.concatenate((symbol_data, conj_symbol_data))
symbol_data = np.insert(symbol_data, 0, 0)
symbol_data = np.insert(symbol_data, 512, 0)
print(symbol_data.shape)
time_symbol_data = np.fft.ifft(symbol_data)
print(time_symbol_data.shape)
time_symbol_data = np.real(time_symbol_data)
time_symbol_data = time_symbol_data / np.max(np.abs(time_symbol_data))
cyclic_data = np.concatenate((time_symbol_data[-32:], time_symbol_data))
print(cyclic_data.shape)

(1024,)
(1024,)
(1024,)
(1024,)
(1056,)


In [24]:
to_transmit = np.concatenate((chirp, cyclic, cyclic_data))
to_transmit = to_transmit.astype(np.float32)
print(to_transmit.shape)
print(np.max(np.abs(to_transmit)))


(46212,)
1.0


In [27]:
generate_sound(to_transmit, 1, fs)

Played sound for 1.08 seconds


In [ ]:
#Signal Import
data = pd.read_csv('data.csv').to_numpy()
data = np.reshape(data, len(data)) 
y = data

In [ ]:
#Synchronization
chirp_time = 1
fs = 44100
t = np.linspace(0, chirp_time, fs*chirp_time)
f_0 = 1000
f_1 = 16000
chirp = signal.chirp(t, f0=f_0, f1=f_1, method='linear')
max_index, cross_correlation, lags = matched_filter_synchronisation(y,chirp, channel_time, fs)
print(max_index)

#Plot
x = np.linspace(0, len(y), len(y))
plt.plot(x[:lags[max_index]],y[:lags[max_index]])
plt.plot(x[lags[max_index]:lags[max_index] + len(chirp)],y[lags[max_index]:lags[max_index] + len(chirp)], color = 'red')
plt.plot(x[lags[max_index] + len(chirp):], y[lags[max_index] + len(chirp):])
plt.show()


In [ ]:
# Channel estimation using the chirp
chirp_data = y[lags[max_index]:lags[max_index] + len(chirp)]
channel_chirp = standered_estimation(chirp, chirp_data, fs)
plt.plot(channel_chirp)
plt.show()

time_channel_chirp = np.fft.ifft(channel_chirp)
plt.plot(time_channel_chirp)
plt.show()

In [ ]:
# Channel estimation Using OFDM
n = 1056
OFDM_DATA = pd.read_csv('OFDM_DATA.csv').to_numpy()
OFDM_DATA = np.reshape(OFDM_DATA, len(OFDM_DATA))
Start_index = lags[max_index] + len(chirp)

# Channel estimation
Symbol_One = OFDM_DATA[Start_index:Start_index + n]
Symbol_One = Symbol_One[32:]
chanel_estimation = standered_estimation(OFDM_DATA, Symbol_One, 1024)
plt.plot(np.abs(chanel_estimation))
plt.show()
time_channel = np.fft.ifft(chanel_estimation)
plt.plot(np.abs(time_channel))
plt.show()
